In [1]:
import mlrun

In [2]:
project = mlrun.get_or_create_project(
    "churn-agents-webinar",
    parameters={
        "base_image" : "mlrun/mlrun",
        "requirements_file" : "requirements.txt",
        "force_build" : False,
        "source" : "v3io:///bigdata/churn.zip"
    }
)

Project Source: v3io:///bigdata/churn.zip
Exporting project as zip archive to v3io:///bigdata/churn.zip...
> 2025-01-08 19:22:25,115 [info] Project loaded successfully: {"project_name":"churn-agents-webinar"}


In [3]:
import pandas as pd

dataset = project.log_dataset(key="churn", df=pd.read_csv("data/churn_dataset_w_sentiment.csv"))
dataset.uri

'store://datasets/churn-agents-webinar/churn#0@4425379f297fd93ce57c7acbca6193999c550c78'

In [4]:
train_run_id = project.run(
    name="main",
    arguments={
        'source_url': dataset.uri,
        'label_column': 'churn',
        'sentiment_column' : "sentiment_label",
        'ordinal_columns' : ["international_plan", "voice_mail_plan"],
        'drop_columns' : ["area_code", "chat_log", "state"],
        'allow_validation_failure': True,
        'model_name' : 'churn_model'
    },
    dirty=True,
    watch=True
)

> 2025-01-08 19:22:27,637 [warning] WARNING!, You seem to have uncommitted git changes, use .push()
> 2025-01-08 19:22:29,068 [info] Pipeline submitted successfully: {"id":"62299e03-0e80-4421-8651-7bdcc3b51e01","pipeline_name":"churn-agents-webinar-main 2025-01-08 19-22-28"}
> 2025-01-08 19:22:29,069 [info] Pipeline run id=62299e03-0e80-4421-8651-7bdcc3b51e01, check UI for progress


> 2025-01-08 19:22:29,123 [info] Started run workflow churn-agents-webinar-main with run id = '62299e03-0e80-4421-8651-7bdcc3b51e01' by kfp engine
> 2025-01-08 19:22:29,124 [info] Waiting for pipeline run completion: {"project":"churn-agents-webinar","run_id":"62299e03-0e80-4421-8651-7bdcc3b51e01"}


uid,start,state,kind,name,parameters,results
...149563cf,Jan 08 19:26:10,completed,run,validate-validate-model,model_path=store://models/churn-agents-webinar/churn_model#1:latest@62299e03-0e80-4421-8651-7bdcc3b51e01label_column=churnallow_validation_failure=True,passed_suite=False
...8b9a7f2f,Jan 08 19:24:50,completed,run,train,label_column=churnmodel_name=churn_model,best_iteration=2accuracy=0.9545454545454546f1_score=0.8604651162790697precision_score=0.8809523809523809recall_score=0.8409090909090909model_uri=store://models/churn-agents-webinar/churn_model#1:latest@62299e03-0e80-4421-8651-7bdcc3b51e01
...37c55d1e,Jan 08 19:24:18,completed,run,validate-validate-train-test-split,label_column=churnallow_validation_failure=True,passed_suite=False
...c1a0a174,Jan 08 19:23:50,completed,run,data-process-data,"label_column=churntest_size=0.1ordinal_columns=['international_plan', 'voice_mail_plan']drop_columns=['area_code', 'chat_log', 'state']sentiment_column=sentiment_label",
...03380ce7,Jan 08 19:23:01,completed,run,validate-validate-data-integrity,label_column=churnallow_validation_failure=True,passed_suite=False
...5db0385e,Jan 08 19:22:36,completed,run,data-get-data,,


In [5]:
serving_fn = project.get_function("serving")

In [6]:
test = mlrun.get_dataitem("store://datasets/churn-agents-webinar/data-process-data_test#0:latest").as_df()
test = test.drop("churn", axis=1)
data = test.head(5).to_dict(orient="split")["data"]

In [7]:
%%time
serving_fn.invoke(
    "/v2/models/churn_model/predict",
    body={
        "inputs": data
    }
)

> 2025-01-08 19:28:45,606 [info] Invoking function: {"method":"POST","path":"http://nuclio-churn-agents-webinar-serving.default-tenant.svc.cluster.local:8080/v2/models/churn_model/predict"}
CPU times: user 12.3 ms, sys: 0 ns, total: 12.3 ms
Wall time: 70.6 ms


{'id': 'ccdab364-e589-4013-b027-76bb64b995ad',
 'model_name': 'churn_model',
 'outputs': [0.010083646332612918,
  0.008936944991110527,
  0.004941389207162068,
  0.01702900397600304,
  0.014068581639357439],
 'timestamp': '2025-01-08 19:28:45.619154+00:00',
 'model_version': 'latest'}